<a href="https://colab.research.google.com/github/taguka/atlas/blob/master/pytorch_universal_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

In [0]:
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip install torchvision

In [0]:
!pip install pretrainedmodels

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [0]:
!pip3 install kaggle
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json



!kaggle competitions download -c human-protein-atlas-image-classification
!unzip -qq train.zip -d train | awk 'BEGIN {ORS=" "} {if(NR%500==0) print "."}'
!unzip -qq test.zip -d test | awk 'BEGIN {ORS=" "} {if(NR%500==0) print "."}'
!rm test.zip
!rm train.zip

In [1]:
!ls 'gdrive/My Drive'

'Colab Notebooks'   opt   output   train


In [2]:
!ls

adc.json    labels.csv	     __pycache__	    test       train.py
dataset.py  mytransforms.py  sample_data	    train      utils.py
gdrive	    opt		     sample_submission.csv  train.csv


In [0]:
# copy opt to drive
#labels.csv
#dataset.py
#models.py
#mytransforms.py
#train.py
#utils.py

In [0]:
cp -R 'gdrive/My Drive/opt' '/content'

In [0]:
!rm 'train.py'

In [0]:
from google.colab import files
files.upload()

In [0]:
DRIVE='/content/gdrive/My Drive'

In [0]:
!python train.py

Train Epoch: 1 [21600/24764 (87%)]  Loss: 0.148970 (0.1824)  Time: 1.813s, 4.413/s  (1.879s, 4.257/s)  Data: 0.008 (0.009)
Train Epoch: 1 [22400/24764 (90%)]  Loss: 0.194807 (0.1819)  Time: 1.836s, 4.357/s  (1.877s, 4.262/s)  Data: 0.016 (0.009)
Train Epoch: 1 [23200/24764 (94%)]  Loss: 0.207304 (0.1814)  Time: 1.819s, 4.399/s  (1.875s, 4.266/s)  Data: 0.009 (0.009)


In [27]:
!ls

adc.json    labels.csv	     sample_data	    train      utils.py
dataset.py  models.py	     sample_submission.csv  train.csv
gdrive	    mytransforms.py  test		    train.py


In [0]:
import os
import cv2
import numpy as np


BASEPATH = '/content/'
PNGPATH = os.path.join(BASEPATH, 'train/')

def find_inputs(folder, types=('.png')):
    inputs = []
    for root, _, files in os.walk(folder, topdown=False):
      for rel_filename in files:
        base, ext = os.path.splitext(rel_filename)
        if  ext.lower() in types:
          abs_filename = os.path.join(root, rel_filename)
          inputs.append((base, abs_filename))
    return inputs


def main():
  png_inputs = find_inputs(PNGPATH, types=('.png',))
  png_stats_red = []
  png_stats_green = []
  png_stats_blue = []
  png_stats_yellow = []
  flags = cv2.IMREAD_GRAYSCALE
  for f in png_inputs:
    img = cv2.imread(f[1],flags)
    mean, std = cv2.meanStdDev(img)
    if f[0].split('.')[0].endswith('red'):
      png_stats_red.append(np.array([mean[::-1] / 255, std[::-1] / 255]))
    if f[0].split('.')[0].endswith('green'):
      png_stats_green.append(np.array([mean[::-1] / 255, std[::-1] / 255]))
    if f[0].split('.')[0].endswith('blue'):
      png_stats_blue.append(np.array([mean[::-1] / 255, std[::-1] / 255]))
    if f[0].split('.')[0].endswith('yellow'):
      png_stats_yellow.append(np.array([mean[::-1] / 255, std[::-1] / 255]))   
  png_vals_red = np.mean(png_stats_red, axis=0)
  png_vals_green = np.mean(png_stats_green, axis=0)
  png_vals_blue = np.mean(png_stats_blue, axis=0)
  png_vals_yellow = np.mean(png_stats_yellow, axis=0)
  print(png_vals_red)
  print(png_vals_green)
  print(png_vals_blue)
  print(png_vals_yellow)


if __name__ == '__main__':
    main()

[[[0.0804419 ]]

 [[0.13000701]]]
[[[0.05262986]]

 [[0.08796628]]]
[[[0.05474701]]

 [[0.1386317 ]]]
[[[0.08270896]]

 [[0.12718021]]]


0.1429575920936828